# Análisis del riesgo de incumplimiento de los prestatarios


# Tabla de Contenidos

* [1 Introducción](#intro) 
* [2 Importación del archivo de datos y análisis de información general.](#capitulo2)
* [3 Exploración de datos](#capitulo3) 
* [4 Transformación de datos](#capitulo4)
* [5 Trabajar con valores ausentes](#capitulo5)
    * [5.1 Restaurar valores ausentes en `total_income`](#capitulo5.1)
    * [5.2 Restaurar valores en `days_employed`](#capitulo5.2)
* [6 Clasificación de datos](#capitulo6)
* [7 Comprobación de hipótesis](#capitulo7)
* [Conclusión general](#conclusion)


## Introducción <a class="anchor" id="intro"></a>

Este es un informe preparado para la división de préstamos de un banco. En el cual se busca establecer si el estado civil, el número de hijos, nivel de ingresos y propósitos de un cliente tienen un impacto en el incumplimiento de pago de un préstamo. El banco ya tiene algunos datos sobre la solvencia crediticia de los clientes.

Las conclusiones de este informe se tendrán en cuenta al crear una **puntuación de crédito** para un cliente potencial. La **puntuación de crédito** se utiliza para evaluar la capacidad de un prestatario potencial para pagar su préstamo.

### Objetivos
Probar las siguientes hipótesis:
1. El número de hijos puede ser el factor causante en la demora del pago de un préstamo. Mientras más hijos tenga una familia, va a generar más gastos, por lo que esto puede llevar al incumplimiento en el pago de un préstamo.
2. El estado civil de un cliente puede influenciar el tiempo de pago de un préstamo. Las personas casadas podrían generar más gastos que una persona soltera y por tanto incumplirían el pago oportuno de un préstamo.
3. El nivel de ingresos de un cliente puede afectar el pago de un préstamo a tiempo. Aquellos clientes que ganan más tienen mejores posibilidades y recursos para pagar un préstamo que clientes con un sueldo bajo.
4. El reembolso del préstamo a tiempo puede estar afectado por los diferentes propósitos que tiene un préstamo. Los diferentes propósitos de un préstamo pueden afectar la demora en el pago de un préstamo, los clientes que buscan invertir en bienes raíces pueden requerir más tiempo para terminar el pago de un préstamo.

## Importación del archivo de datos y análisis de información general. <a class="anchor" id="capitulo2"></a>

Se inicia importando la librería `pandas`, la cual permitirá abrir el archivo de datos a través del método `.read_csv`. El archivo se almacena en `/datasets/credit_scoring_eng.csv` y se guardará en la variable `data`. Se ha escogido esta librería ya que brinda las herramientas necesarias para procesar datasets y solucionar problemas con los datos.


In [1]:
import pandas as pd

data = pd.read_csv('datasets/credit_scoring_eng.csv')

## Exploración de datos <a class="anchor" id="capitulo3"></a>

Para explorar nuestros datos primero se procede a utilizar el método `shape` para saber el número de filas y columnas con las que se va a trabajar, y se utilizará el método `head` para realizar un análisis exploratorio de los datos. Nuestro dataset cuenta con las siguientes columnas:

**Descripción de los datos**
- `children` - el número de hijos en la familia
- `days_employed` - experiencia laboral en días
- `dob_years` - la edad del cliente en años
- `education` - la educación del cliente
- `education_id` - identificador de educación
- `family_status` - estado civil
- `family_status_id` - identificador de estado civil
- `gender` - género del cliente
- `income_type` - tipo de empleo
- `debt` - ¿había alguna deuda en el pago de un préstamo?
- `total_income` - ingreso mensual
- `purpose` - el propósito de obtener un préstamo


In [2]:
data.shape

(21525, 12)

In [3]:
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


Se registran un total de 12 columnas y 21525 filas. Se puede observar que en la columna `days_employed` existen valores negativos para la experiencia laboral en días, lo que lleva a cuestionarse si hubo algún error al importar el dataset o algún error técnico o humano, se necesitará investigar un poco más estos datos antes de procesarlos. 

También se puede observar duplicados en la columna `education`, los cuales van a necesitar ser gestionados de la manera correcta para evitar problemas posteriores. En dos columnas del dataframe aparecen valores ausentes, los cuáles van a necesitar un análisis más detallado, para esto aplicaremos el método `info` a nuestros datos. 

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Existen valores ausentes en dos de las doce columnas de nuestro dataset, tanto la columna `days_employed` y la columna `total_income` tienen el mismo número de valores ausentes, lo que nos lleva a cuestionarnos si quizás estos valores ausentes están relacionados con si el cliente se encuentra o no trabajando. Al estar desempleado o retirado no genera ingresos, de ahí podrían surgir esos valores ausentes. Se debería investigar más a fondo para poder analizar realizar un análisis correcto de estos valores.

Para analizar las dos columnas con valores ausentes en primer lugar filtramos el dataframe por la columna `days_employed` y observamos el número de valores ausentes, luego para poder establecer si los valores son simétricos se aplica un filtrado consecutivo.

In [5]:
# Tabla filtrada por valores ausentes en la columna days_employed
data_filtered = data[data['days_employed'].isna()]
data_filtered

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


In [6]:
#Tabla filtrada por las dos columnas que presentan valores ausentes
data_filtered = data_filtered[data_filtered['total_income'].isna()] 
data_filtered

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


A través de la tabla filtrada `data_filtered`se observa que el número de filas con valores ausentes en la columna `days_employed` es 2174, y al parecer los valores ausentes coinciden con los valores ausentes de la columna `total_income`. Esto se comprueba al realizar un filtrado consecutivo por la columna `total_income`, ya que el número de filas se mantiene en el mismo valor de 2174. Con esto se puede corroborar que los valores ausentes dentro de nuestro dataframe presentan simetría.

Para estar más seguros que nuestros valores ausentes son simétricos realizamos un conteo de los valores ausentes en las dos columnas de la tabla filtrada `data_filtered`.

In [7]:
# Conteo de valores ausentes.
data_filtered[['days_employed', 'total_income']].isna().sum()

days_employed    2174
total_income     2174
dtype: int64

**Conclusión intermedia**

La tabla filtrada por la columna con valores ausentes `days_employed` presenta un total de 2174 filas, al realizar un filtrado consecutivo por la segunda columna con valores ausentes `total_income` se obtiene el mismo número de filas. Posteriormente se confirma que el número de valores ausentes en nuestra tabla filtrada `data_filtered` es de 2174. Pudiendo concluir que el número de filas en nuestra tabla filtrada coincide con el número de valores ausentes, y comprobando entonces que existe simetría en los valores ausentes de nuestro dataframe. 

Después de estas conclusiones, se confirma que existen cerca de un 10% de valores ausentes en nuestro dataframe, este porcentaje es grande por lo que será necesario rellenar estos valores y también hacer un análisis más profundo sobre si nuestros valores NaN están relacionados con otras variables o características específicas del cliente. Por ejemplo, podría ser que el tipo de empleo influya en el número de valores ausentes, aquellos clientes retirados no van a generar ingresos y serían la causa de estos valores en nuestro dataframe. También podría estar relacionado con el nivel de educación y la dificultad para conseguir empleo o hasta incluso la edad o el género del cliente. 

Para poder establecer una correlación entre los valores ausentes y otras características del cliente, vamos a comparar la distribución de las variables entre nuestro dataframe con y sin valores ausentes, de esta manera se confirmára o no la presencia de un patrón que influya en estos valores. 

Es así que se procedió a eliminar los valores ausentes de nuestro dataframe, para esto se utilizó el método `dropna` y se guardaron estos datos filtrados en la variable `data_without_nan`. 

In [8]:
# Tabla sin valores ausentes
data_without_nan = data.dropna()
data_without_nan

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21522,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


In [9]:
# Distribución en tabla sin valores ausentes.
for column in data_without_nan:
    print(data_without_nan[column].value_counts(normalize = True))

 0     0.656814
 1     0.224433
 2     0.095654
 3     0.015193
 20    0.003462
-1     0.002274
 4     0.001757
 5     0.000413
Name: children, dtype: float64
-8437.673028      0.000052
-3507.818775      0.000052
 354500.415854    0.000052
-769.717438       0.000052
-3963.590317      0.000052
                    ...   
-1099.957609      0.000052
-209.984794       0.000052
 398099.392433    0.000052
-1271.038880      0.000052
-1984.507589      0.000052
Name: days_employed, Length: 19351, dtype: float64
35    0.028577
41    0.028319
38    0.028112
40    0.028061
34    0.027595
42    0.027492
33    0.027389
39    0.026975
44    0.025993
29    0.025580
31    0.025580
48    0.025425
36    0.025425
37    0.025012
30    0.024908
32    0.024443
43    0.023926
50    0.023926
49    0.023668
27    0.023616
45    0.023100
28    0.023048
56    0.022376
52    0.022273
46    0.022066
54    0.021911
47    0.021756
53    0.021446
59    0.021188
58    0.020929
57    0.020877
51    0.020567
55    0.02041

**Posibles razones por las que hay valores ausentes en los datos**

Al analizar nuestro dataframe filtrado sin valores ausentes, parece que no existe un patrón en la distribución de valores ausentes, quizás los valores ausentes se deban a factores del azar, simplemente estos valores están presentes porque los clientes están desempleados y no generan ingresos. Pero antes de sacar conclusiones, comparemos estos resultados con nuestro dataframe completo.  


In [10]:
# Distribución de datos en el dataframe con valores ausentes
for column in data:
    print(data[column].value_counts(normalize = True))

 0     0.657329
 1     0.223833
 2     0.095470
 3     0.015331
 20    0.003531
-1     0.002184
 4     0.001905
 5     0.000418
Name: children, dtype: float64
-8437.673028      0.000052
-3507.818775      0.000052
 354500.415854    0.000052
-769.717438       0.000052
-3963.590317      0.000052
                    ...   
-1099.957609      0.000052
-209.984794       0.000052
 398099.392433    0.000052
-1271.038880      0.000052
-1984.507589      0.000052
Name: days_employed, Length: 19351, dtype: float64
35    0.028664
40    0.028293
41    0.028200
34    0.028014
38    0.027782
42    0.027735
33    0.026992
39    0.026620
31    0.026016
36    0.025784
44    0.025412
29    0.025319
30    0.025087
48    0.024994
37    0.024948
50    0.023879
43    0.023833
32    0.023693
49    0.023600
28    0.023368
45    0.023089
27    0.022904
56    0.022625
52    0.022485
47    0.022300
54    0.022253
46    0.022067
58    0.021417
57    0.021370
53    0.021324
51    0.020813
59    0.020627
55    0.02058

**Conclusión intermedia**

Al comparar la distribución de nuestros datos con valores y sin valores ausentes, se observa que no existe una diferencia significativa en la distribución de las diferentes variables. La única variable en la cual la distribución de datos varía es en la edad de los clientes. En la tabla sin valores ausentes `data_without_nan` se puede observar que los clientes de 35 años representan el 2,8% de los datos, seguidos de clientes de 41, 38 y 40 años, mientras que en el dataframe completo esta distribución cambia siendo los clientes 35, 40, 41 y 34 años los que representan el 11,2% de los datos de la variable `dob_years`. 

Anteriomente, se mencionó que el tipo de empleo influiría en esto valores ausentes, pero al comparar la distribución entre la tabla filtrada sin valores ausentes y la tabla con valores ausentes, no existe una diferencia significativa, siendo employee la categoría con mayor porcentaje de clientes, representando el 51% en ambas tablas. De esta manera también se pueda negar el hecho de que los clientes retirados o desempleados influyen sobre los valores ausentes. 

**Conclusiones**

Se concluye entonces que nuestros valores ausentes no presentan un patrón subyacente ya que la distribución de las variables no presenta mayor diferencia entre nuestra base de datos con y sin valores ausentes. Pudiendo confirmar que nuestros datos son aleatorios. 

En base a estos resultados será necesario abordar los valores ausentes. En el caso de la columna `total_income`, los valores NaN pueden estar relacionados con el nivel de educación del cliente o el tipo de empleo, aquellas personas graduadas de la universidad tienen acceso a mejores salarios o un servidor público puede generar más ingresos que un estudiante. Así que se considerará estos dos factores para rellenar los valores ausentes. Por otro lado, en la columna `days_employed` los valores ausentes pueden también estar relacionados con el tipo de empleo o incluso con la edad del cliente, a menor edad menor número de días que una persona se encuentra trabajando. Se completará estos valores con cualquiera de las variables mencionadas después de analizar los valores medios y medianos de nuestros datos sin valores ausentes. 

Después de analizar los valores ausentes, es necesario abordar datos incorrectos, como observamos en la columna `days_employed`, existían valores negativos y un número de días de trabajo irreal. En consecuencia, será necesario analizar cada variable y corregir cualquier tipo de error, como lo vamos a hacer a continuación.




## Transformación de datos<a class="anchor" id="capitulo4"></a>

Antes de analizar nuestros datos para responder a nuestras hipótesis, es necesario transformar ciertos valores en el dataframe para evitar problemas futuros. Para esto vamos a repasar los valores únicos en cada una de las 12 columnas presentes e ir corrigiendo posibles errores como duplicados, registros diferentes, artefactos incorrectos y valores ausentes. 

Vamos a empezar analizando la lista de valores únicos en la columna `education`. Al aplicar el método `value_counts()` se puede observar que existen duplicados pero no son reconocidos como tales ya que las cadenas no son idénticas. En este caso utilizamos el método `str.lower()` que transformará todas las cadenas en minúsculas y eliminará los duplicados. 

In [11]:
# Registros únicos con el método value_counts
data['education'].value_counts()

secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
Graduate Degree            1
GRADUATE DEGREE            1
Name: education, dtype: int64

In [12]:
# Se arregla los registros con el método str.lower
data['education'] = data['education'].str.lower()

In [13]:
# Se comprueba la eliminación de duplicados
data['education'].value_counts()

secondary education    15233
bachelor's degree       5260
some college             744
primary education        282
graduate degree            6
Name: education, dtype: int64

En segundo lugar, vamos a analizar los valores únicos de la columna `children`.

In [14]:
# Veamos la distribución de los valores en la columna `children`
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

En la columna `children` se puede encontrar dos valores inusuales 20 y -1, quizás estos valores incorrectos se ocasionaron por errores humanos al momento de ingresar los datos, ya que es inusual encontrar familias con 20 hijos. Puede ser muy probable que -1 sera un error humano al tratar de ingresar 1, y 20 sea un error al tratar de ingresar 2. Estos errores corresponden a menos del 1% de datos, pero va a ser necesario arreglarlos para evitar problemas al momento de analizar nuestros datos. Para esto vamos a reemplazar los valores 20 con 2 y -1 con 1. 

In [15]:
# Reemplazo de valores 20 y -1 con valores correspondientes
data.loc[data['children'] == 20, 'children'] = 2
data.loc[data['children'] == -1, 'children'] = 1

In [16]:
# Se comprueba si se eliminaron los duplicados
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Ahora vamos a buscar datos problemáticos en la columna `days_employed`.

In [17]:
data['days_employed'].value_counts()

-8437.673028      1
-3507.818775      1
 354500.415854    1
-769.717438       1
-3963.590317      1
                 ..
-1099.957609      1
-209.984794       1
 398099.392433    1
-1271.038880      1
-1984.507589      1
Name: days_employed, Length: 19351, dtype: int64

In [18]:
data[data['days_employed'] < 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
...,...,...,...,...,...,...,...,...,...,...,...,...
21519,1,-2351.431934,37,graduate degree,4,divorced,3,M,employee,0,18551.846,buy commercial real estate
21520,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21522,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car


Al observar la columna `days_employed` nos encontramos con valores float negativos, al filtrar esta tabla por aquellos valores menores a 0, se confirma que 15906 filas presentan este tipo de datos, lo que indica que el 73,89% de datos en esta columna no son correctos. En este caso los valores negativos pueden estar relacionados a que se restó la fecha de inicio de trabajo de la fecha actual, y al importar se produjo un error. Para corregirlos, vamos a aplicar el método abs() para obtener el valor absoluto, y transformaremos los valores NaN en 0 para poder transformar los valores float en números enteros por el método astype(). Escogemos esta metodología, ya que NaN es un valor float que no admite ser convertido en valor entero, al transformarlo a 0, se puede realizar la conversión a este tipo de dato con mayor facilidad. 

In [19]:
# Se obtiene el valor absoluto y entero, y se rellenan los valores NaN con cero
data['days_employed'] = data['days_employed'].abs().fillna(0).astype('int')

In [20]:
# Se comprueba que los valores son correctos
data['days_employed']

0          8437
1          4024
2          5623
3          4124
4        340266
          ...  
21520      4529
21521    343937
21522      2113
21523      3112
21524      1984
Name: days_employed, Length: 21525, dtype: int32

Una vez corregidos los valores negativos de la columna `days_employed` es necesario abordar los valores atípicos de más de 300.000 días de trabajo. Quizás estos valores también son producto de un error al momento de importar la base de datos. Para poder tomar una decisión correcta sobre qué hacer con estos valores, vamos a encontrar patrones que estarían influenciando sobre estos valores. 

En primer lugar, consideraremos que una persona trabaja un promedio de 40 años en su vida, lo que serían 14,600 días si trabajara los 365 días del año. Así que vamos a filtrar nuestra tabla por aquellos valores superiores a 20,000 días, para ver si estos valores exhiben algún patrón. 

In [21]:
data_days = data[data['days_employed'] > 20000]
data_days

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
18,0,400281,53,secondary education,1,widow / widower,2,F,retiree,0,9091.804,buying a second-hand car
24,1,338551,57,secondary education,1,unmarried,4,F,retiree,0,46487.558,transactions with commercial real estate
25,0,363548,67,secondary education,1,married,0,M,retiree,0,8818.041,buy real estate
30,1,335581,62,secondary education,1,married,0,F,retiree,0,27432.971,transactions with commercial real estate
...,...,...,...,...,...,...,...,...,...,...,...,...
21505,0,338904,53,secondary education,1,civil partnership,1,M,retiree,0,12070.399,to have a wedding
21508,0,386497,62,secondary education,1,married,0,M,retiree,0,11622.175,property
21509,0,362161,59,bachelor's degree,0,married,0,M,retiree,0,11684.650,real estate transactions
21518,0,373995,59,secondary education,1,married,0,F,retiree,0,24618.344,purchase of a car


Al parecer, la mayoría de estos datos están relacionados con la variable `income_type`, siendo la categoría `retiree` la que almacena estos valores atípicos. Pero primero provemos si solo la categoría `retiree` presenta estos datos anómalos. Filtremos nuestra tabla por aquellos valores que no corresponden a la categoría `retiree` y veamos el resultado.

In [22]:
data_days = data[(data['days_employed'] > 20000) & (data['income_type'] != 'retiree')]
data_days

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
3133,1,337524,31,secondary education,1,married,0,M,unemployed,1,9593.119,buying property for renting out
14798,0,395302,45,bachelor's degree,0,civil partnership,1,F,unemployed,0,32435.602,housing renovation


In [23]:
data[(data['days_employed'] == 0) & (data['income_type'] == 'retiree')]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,0,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
29,0,0,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
55,0,0,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
67,0,0,52,bachelor's degree,0,married,0,F,retiree,0,NaN,purchase of the house for my family
145,0,0,62,secondary education,1,married,0,M,retiree,0,NaN,building a property
...,...,...,...,...,...,...,...,...,...,...,...,...
21311,0,0,49,secondary education,1,married,0,F,retiree,0,NaN,buying property for renting out
21321,0,0,56,secondary education,1,married,0,F,retiree,0,NaN,real estate transactions
21414,0,0,65,secondary education,1,married,0,F,retiree,0,NaN,purchase of my own house
21415,0,0,54,secondary education,1,married,0,F,retiree,0,NaN,housing transactions


In [24]:
data_retiree = data[data['income_type'] == 'retiree']
data_retiree['income_type'].count()

3856

In [25]:
data_retiree_days = data_retiree[data_retiree['days_employed'] > 20000]
data_retiree_days['days_employed'].count()

3443

In [26]:
data_retiree_zero = data_retiree[data_retiree['days_employed'] == 0]
data_retiree_zero['days_employed'].count()

413

Aparentemente, los valores anómalos también se presentan en la categoría `unemployed`. Lo más lógico sería reemplazar estos valores por los promedios de la columna `income_type`, pero si realizamos un análisis más a fondo como se presenta en las variables `data_retiree_days` y `data_retiree_zero`, nos encontramos que de los 3856 clientes que corresponden a la caterogía retiree, 3443 presentan valores fuera del rango normal de días de trabajo y los otros 413 son los valores NaN que fueron reemplazados por 0. Así mismo, los dos únicos clientes en la categoría `unemployed`presentan un número anómalo de días de trabajo. Es así que no se podrá imputar estos valores con la media por el tipo de ingreso, ya que no tenemos un valor para estas dos categorías.

Si tomamos todo esto en consideración, lo mejor va a ser reemplazar estos valores por 0 y posteriormente analizaremos más a fondo que variable va a ser la más adecuada para rellenar estos valores ausentes en la columna `days_employed`. Comprobamos que hemos reemplazado estos valores al imprimir el dataframe filtrado por los valores mayores a 20000, y obteniendo una tabla vacía.

In [27]:
data.loc[data['days_employed'] > 20000, 'days_employed'] = 0
data.loc[data['days_employed'] > 20000] 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Ahora vamos a analizar la columna correspondiente a la edad de los clientes `dob_years`:

In [28]:
# Revisa `dob_years` en busca de valores sospechosos y cuenta el porcentaje
data['dob_years'].value_counts().sort_index()

0     101
19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    513
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64

En la columna `dob_years` se presentan 101 valores correspondientes a la edad 0, en este caso lo mejor sería rellenar los valores con el valor de la media o la mediana de edades en esta columna. Para esto procedemos a calcular a través del método agg() los valores promedios de edades de acuerdo al género. Al no observar una diferencia significativa, procedemos a rellenar los valores en 0 con la media de la columna `dob_years`.

In [29]:
mean_gender = data.groupby('gender').agg({'dob_years': ['mean', 'median']})
mean_gender

dob_years       
             mean median
gender                  
F       44.471972   44.0
M       40.993825   40.0
XNA     24.000000   24.0

In [30]:
mean_age = int(data['dob_years'].mean())
data.loc[data['dob_years'] == 0, 'dob_years'] = mean_age 

In [31]:
# Se comprueba si se eliminaron valores cero
data['dob_years'].value_counts().sort_index()

19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    614
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64

Ahora vamos a revisar la columna `family_status` para encontrar valores problemáticos.

In [32]:
data['family_status'].value_counts()

married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64

Como podemos observar no existen problemas en la columna `family_status` así que no es necesario realizar ningún tipo de corrección.

Al no existir valores problemáticos en la columna anterior procedemos a analizar la siguiente columna `gender`. En esta columna se observa el valor de género categórico XNA, que quizás se debió a un error al momento de ingresar los datos o a que el cliente no facilitó esos datos. En este caso lo mejor sería asignar a este valor desconocido el de la categoría de género más común, la cual pertenece al género femenino.

In [33]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [34]:
data.loc[data['gender'] == 'XNA', 'gender'] = 'F'

In [35]:
data['gender'].value_counts()

F    14237
M     7288
Name: gender, dtype: int64

Finalmente analizaremos la columna `income_type`, en la cual al aplicar el método value_counts no se observa ningún tipo de dato problemático. 

In [36]:
data['income_type'].value_counts()

employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

Antes de finalizar la transformación de nuestros datos, es importante analizar los valores duplicados presentes en nuestro dataframe para evitar problemas futuros. Para esto aplicaremos el método `duplicated()` para encontrar duplicados en nuestro dataframe y posteriormente utilizaremos `drop_duplicates()` para eliminar estos duplicados.

In [37]:
data.duplicated().sum()

71

In [38]:
data = data.drop_duplicates().reset_index(drop = True)

In [39]:
data.duplicated().sum()

0

In [40]:
data.shape

(21454, 12)

Nuestro nuevo conjunto de datos contiene 21454 filas y 12 columnas, 71 filas del dataframe fueron eliminadas ya que contenían duplicados. A su vez, se reemplazaron los valores duplicados de la columna `education` por su equivalente en letras minúsculas, y se cambiaron los valores inusuales de la columna `children` por el promedio del número de hijos. En la columna `days_employed` se transformaron los números reales negativos por números enteros debido a un probable error al importar los datos, y se reemplazaron los valores anómalos y NaN por cero. Por otro lado, en la columna `dob_years` se registró el valor 0, que fue modificado y reemplazado por la edad promedio de los clientes. En la columna `gender` se encontró un valor incorrecto, el cual fue sustituido por la categoría con mayor número de valores, en este caso fue el género femenino. En la columna income_type no se encontraron valores insusuales.

En conclusión, en la columnas `children`, `gender` y `dob_years` se cambiaron menos del 1% de los valores, mientras que en la columna `education` se reemplazaron el 9% de los datos. La columna con mayor porcentaje de cambio fue la columna `days_employed`, en la que el 73% de los datos fueron modificados. El porcentaje de valores duplicados eliminados fue del 0,32%.

## Trabajar con valores ausentes <a class="anchor" id="capitulo5"></a>

Para poder reemplazar los valores ausentes en las columnas de nuestro dataframe, vamos a utilizar diccionarios para los dos columnas que presentan valores ausentes y se aplicará la función mean() y median() a cada una de estas. El diccionario para `total_income` se guardará en la variable `total_income_func` y el diccionario para `days_employed` se guardará en `days_employed_func`.

In [41]:
# Diccionarios que se utilizarán
total_income_func = {'total_income': ['mean', 'median', 'max', 'min']}
days_employed_func = {'days_employed': ['mean', 'median', 'max', 'min']}

### Restaurar valores ausentes en `total_income` <a class="anchor" id="capitulo5.1"></a>

Dentro de nuestro dataframe se pueden encontrar dos columnas con valores ausentes: la columna `total_income` y la columna `days_employed`. Para la columna `total_income`, los valores ausentes serán rellenados con los valores promedios o medianas de acuerdo al nivel de educación del cliente o al tipo de ingreso. En el caso de `days_employed` se rellenará con las medias o medianas de acuerdo a la edad, ya que no se podrá utilizar el tipo de ingreso, porque los valores de la categoría retiree fueron reemplazados por cero al presentar valores atípicos.


Para poder rellenar los valores ausentes de la columna `total_income` vamos primero a definir categorías por edades. La columna `income_type` presenta edades que varían desde los 19 a los 75 años. Para construir categorías se recomienda que los rangos sean lógicos, que sean del mismo tamaño a excepción del primer y último rango, y que no proporcionen demasiada información que luego será difícil de procesar. En este caso vamos a optar por formar grupos de 10 años, y también consideraremos edades inferiores a los 19 años, ya que una buena función puede tomar cualquier valor de edad y devolver el rango al que pertenece. Posteriormente, construiremos la función `assing_age_group` que nos permitirá establecer las categorías de edad y se creará la columna `age_group` que resultará de aplicar la función a nuestra columna `dob_years`.

In [42]:
def assing_age_group(age):
    if age < 0 or pd.isna(age):
        return 'NA'
    elif age < 10:
        return '0-9'
    elif age >= 10 and age<=19:
        return '10-19'
    elif age >= 20 and age<=29:
        return '20-29'
    elif age >= 30 and age<=39:
        return '30-39'
    elif age >= 40 and age<=49:
        return '40-49'
    elif age >= 50 and age<=59:
        return '50-59'
    elif age >= 60 and age<=69:
        return '60-69'
    else:
        return '70+'

In [43]:
# Comprobamos si la funcion trabaja correctamente
assing_age_group(data.loc[0, 'dob_years'])

'40-49'

In [44]:
# Se crea la columna age_group
data['age_group'] = data['dob_years'].apply(assing_age_group)

In [45]:
data['age_group']

0        40-49
1        30-39
2        30-39
3        30-39
4        50-59
         ...  
21449    40-49
21450    60-69
21451    30-39
21452    30-39
21453    40-49
Name: age_group, Length: 21454, dtype: object

Una vez creada la nueva columna `age_group` procedemos a decidir con que valores van a ser rellenados nuestros valores ausentes en la columna `total_income`, para esto construiremos un dataframe sin valores NaN y los guardaremos en la variable `data_no_missing_values`. Posteriormente realizaremos un análisis de las diferentes variables que pueden influir en los ingresos totales. Por lo tanto, analizaremos las variables `age_group`, `education`, `income_type` y `gender`.

In [46]:
data_no_missing_values = data.dropna()
data_no_missing_values.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-49
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,30-39
2,0,5623,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,30-39
3,3,4124,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,30-39
4,0,0,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,50-59
5,0,926,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,20-29
6,0,2879,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,40-49
7,0,152,50,secondary education,1,married,0,M,employee,0,21731.829,education,50-59
8,2,6929,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,30-39
9,0,2188,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,40-49


In [47]:
income_by_age = data_no_missing_values.pivot_table(index='age_group', values='total_income', aggfunc=total_income_func)
income_by_age

,max,mean,median,min
age_group,,,,
10-19,26753.823,16993.942462,14934.9010,9459.851
20-29,131588.163,25572.630177,22799.2580,4494.861
30-39,352136.354,28312.479963,24667.5280,3392.845
40-49,362496.645,28491.929026,24755.6960,4036.463
50-59,195686.797,25811.700327,22203.0745,3306.762
60-69,274402.943,23242.812818,19817.4400,3471.216
70+,57508.032,20125.658331,18751.3240,3595.641


In [48]:
income_by_education = data_no_missing_values.pivot_table(index='education', values='total_income', aggfunc=total_income_func)
income_by_education

,max,mean,median,min
education,,,,
bachelor's degree,362496.645,33142.802434,28054.5310,5148.514
graduate degree,42945.794,27960.024667,25161.5835,15800.399
primary education,78410.774,21144.882211,18741.9760,4049.374
secondary education,276204.162,24594.503037,21836.5830,3306.762
some college,153349.533,29045.443644,25618.4640,5514.581


In [49]:
income_by_gender = data_no_missing_values.pivot_table(index='gender', values='total_income', aggfunc=total_income_func)
income_by_gender

,max,mean,median,min
gender,,,,
F,274402.943,24656.229647,21465.165,3306.762
M,362496.645,30907.144369,26834.295,3392.845


In [50]:
income_by_type = data_no_missing_values.pivot_table(index='income_type', values='total_income', aggfunc=total_income_func)
income_by_type

,max,mean,median,min
income_type,,,,
business,362496.645,32386.793835,27577.2720,4592.450
civil servant,145672.235,27343.729582,24071.6695,4672.012
employee,276204.162,25820.841683,22815.1035,3418.824
entrepreneur,79866.103,79866.103000,79866.1030,79866.103
paternity / maternity leave,8612.661,8612.661000,8612.6610,8612.661
retiree,117616.523,21940.394503,18962.3180,3306.762
student,15712.260,15712.260000,15712.2600,15712.260
unemployed,32435.602,21014.360500,21014.3605,9593.119


Al comparar los valores de media, mediana, max y min en la columna `total_income` se puede llegar a la conclusión que el nivel de educación y el tipo de ingreso están influenciando en los ingresos totales. En el caso del tipo de ingreso aquellos clientes que son estudiantes, retirados o tiene permiso de maternidad/paternidad reciben menos ingresos que aquellos que trabajan en negocios o son servidores públicos. Por otro lado, si nos fijamos en la tabla `income_by_education`, las personas con educación primaria reciben en promedio un ingreso mucho menor a clientes que han tenido educación universitaria.

En base a esto construimos una tabla dinámica agrupada por las categorías de tipo de ingreso y educación, la cual se guardará en la variable `income_type_education`. Al analizar más a fondo esta tabla, podemos observar que los valores de medias y medianas no presentan diferencias significativas. Por lo que optaremos por utilizar el valor de la media para rellenar nuestros valores ausentes de la columna `total_income`. 

In [51]:
income_type_education = data_no_missing_values.pivot_table(index=['education', 'income_type'], values='total_income', aggfunc=total_income_func)
income_type_education

max          mean  \
education           income_type                                             
bachelor's degree   business                     362496.645  38780.136881   
                    civil servant                145672.235  31571.287664   
                    employee                     205804.960  30650.288996   
                    entrepreneur                  79866.103  79866.103000   
                    retiree                      117616.523  27306.878056   
                    student                       15712.260  15712.260000   
                    unemployed                    32435.602  32435.602000   
graduate degree     civil servant                 17822.757  17822.757000   
                    employee                      42945.794  31089.653667   
                    retiree                       40868.031  28334.215000   
primary education   business                      60576.988  26409.124931   
                    civil servant                 78410.774  29449.016667   
                    employee                      65263.983  21954.056075   
                    retiree                       47385.648  17810.387914   
secondary education business                     255618.158  28718.435242   
                    civil servant                 83412.012  24648.816597   
                    employee                     276204.162  24426.079549   
                    paternity / maternity leave    8612.661   8612.661000   
                    retiree                      109008.094  21071.829349   
                    unemployed                     9593.119   9593.119000   
some college        business                     153349.533  31623.893705   
                    civil servant                 82285.319  27596.312587   
                    employee                     131588.163  27951.531586   
                    retiree                       65981.816  22129.937314   

                                                     median        min  
education           income_type                                         
bachelor's degree   business                     32285.6640   6844.452  
                    civil servant                27601.7775   7157.482  
                    employee                     26502.5190   5148.514  
                    entrepreneur                 79866.1030  79866.103  
                    retiree                      23078.5230   5622.079  
                    student                      15712.2600  15712.260  
                    unemployed                   32435.6020  32435.602  
graduate degree     civil servant                17822.7570  17822.757  
                    employee                     31771.3210  18551.846  
                    retiree                      28334.2150  15800.399  
primary education   business                     21887.8250  11368.245  
                    civil servant                23734.2870   8495.926  
                    employee                     20159.1860   5837.099  
                    retiree                      16415.7850   4049.374  
secondary education business                     25451.3100   4592.450  
                    civil servant                21864.4750   4672.012  
                    employee                     21848.8175   3418.824  
                    paternity / maternity leave   8612.6610   8612.661  
                    retiree                      18374.8570   3306.762  
                    unemployed                    9593.1190   9593.119  
some college        business                     28778.7440   8514.371  
                    civil servant                25694.7750   8154.029  
                    employee                     24209.4300   8163.589  
                    retiree                      19221.9030   5514.581

Una vez que se ha decidido utilizar la media para rellenar los valores ausentes en la columna `total_income` procedemos a construir la función `fill_nan`. La cual se aplicará a la columna `total_income` a través del método apply para finalmente reemplazar los valores ausentes.

In [52]:
income_by_education = data_no_missing_values.pivot_table(index='education', values='total_income', aggfunc=total_income_func)
income_by_education

,max,mean,median,min
education,,,,
bachelor's degree,362496.645,33142.802434,28054.5310,5148.514
graduate degree,42945.794,27960.024667,25161.5835,15800.399
primary education,78410.774,21144.882211,18741.9760,4049.374
secondary education,276204.162,24594.503037,21836.5830,3306.762
some college,153349.533,29045.443644,25618.4640,5514.581


In [53]:
def fill_nan(row):
    income_type = row['income_type']
    education = row['education']
    total_income = row['total_income']
    
    if pd.isna(total_income):
        return income_type_education.loc[(education, income_type), 'mean']
    return total_income

In [54]:
# Comprobamos si funciona la función fill_nan
fill_nan(data.iloc[12])

21071.82934944504

In [55]:
# Comprobamos que no existan errores al aplicar la función
try:
    data['total_income'] = data.apply(fill_nan, axis=1)
except:
    print('Error')

In [56]:
data['total_income'].isna().sum()

0

In [57]:
data.count()

children            21454
days_employed       21454
dob_years           21454
education           21454
education_id        21454
family_status       21454
family_status_id    21454
gender              21454
income_type         21454
debt                21454
total_income        21454
purpose             21454
age_group           21454
dtype: int64

Una vez reemplazados los valores ausentes, procedemos a comprobar si existe un error en nuestra función `fill_nan` para esto utilizaremos un bloque try-except, el cual nos confirma que la función está funcionando correctamente. Esto se comprueba al pasar el método `isna()` y `sum()` a la columna ingresos totales. Finalmente, se confirma al llamar al método count que ya no existen valores ausentes en la columna `total_income`, ya que el número de filas es similar al de otras columnas del dataframe.

###  Restaurar valores en `days_employed` <a class="anchor" id="capitulo5.2"></a>

Para restaurar los valores ausentes en la columna `days_employed`, se realizará un análisis de los diferentes parámetros que pueden repercutir en estos valores. Así que al igual que en la columna anterior, compararemos diferentes métricas en las variables `age_group`, `gender`, `income_type`, `education`. A partir de esto decidiremos que variables utilizaremos para reemplazar los valores ausentes. Recordemos que se reemplazaron los valores NaN por los valores 0, y posteriormente se cambiaron los valores anómalos en la columna `days_employed` por valores 0, así que debemos considerar que métrica nos ayudará a corregir estos datos.

Empezaremos por crear la tabla filtrada sin valores cero `data_no_zeros`, a partir de la cual crearemos tablas dinámicas que nos permitan entender mejor que parámetros son los que están influyendo en los valores ausentes de esta columna.

In [58]:
data_no_zeros = data[data['days_employed'] != 0]
data_no_zeros

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-49
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,30-39
2,0,5623,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,30-39
3,3,4124,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,30-39
5,0,926,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,20-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21448,1,2351,37,graduate degree,4,divorced,3,M,employee,0,18551.846,buy commercial real estate,30-39
21449,1,4529,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,40-49
21451,1,2113,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,30-39
21452,3,3112,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,30-39


A partir de nuestra tabla sin valores cero, procedemos a analizar las diferentes variables que pueden impactar los datos del número de días de trabajo

In [59]:
days_by_income = data_no_zeros.pivot_table(index='income_type', values='days_employed', aggfunc=days_employed_func)
days_by_income

,max,mean,median,min
income_type,,,,
business,17615,2111.031899,1547.0,30
civil servant,15193,3399.400915,2689.0,39
employee,18388,2325.996904,1573.5,24
entrepreneur,520,520.000000,520.0,520
paternity / maternity leave,3296,3296.000000,3296.0,3296
student,578,578.000000,578.0,578


In [60]:
days_by_education = data_no_zeros.pivot_table(index='education', values='days_employed', aggfunc=days_employed_func)
days_by_education

,max,mean,median,min
education,,,,
bachelor's degree,16593,2277.314660,1611.0,24
graduate degree,5968,3520.000000,3851.5,409
primary education,11761,1940.160714,1189.0,62
secondary education,18388,2433.774291,1684.0,24
some college,13264,1562.603125,1148.0,51


In [61]:
days_by_gender = data_no_zeros.pivot_table(index='gender', values='days_employed', aggfunc=days_employed_func)
days_by_gender

,max,mean,median,min
gender,,,,
F,18388,2518.667303,1748.5,24
M,15267,2075.246477,1437.5,24


In [62]:
days_by_age = data_no_zeros.pivot_table(index='age_group', values='days_employed', aggfunc=days_employed_func)
days_by_age

,max,mean,median,min
age_group,,,,
10-19,1020,633.153846,724.0,111
20-29,5673,1212.042947,999.0,51
30-39,9762,2025.908447,1589.0,24
40-49,13210,2724.485983,2009.0,30
50-59,16264,3260.332587,2261.0,58
60-69,18388,3838.694268,2669.0,100
70+,13923,4226.200000,2679.5,486


Al analizar las diferentes variables que pueden generar patrones en los valores ausentes de la columna `days_employed`, podemos concluir que uno de los parámetros que influye en nuestros datos es la edad. Es así que a más edad, el promedio del número de días de trabajo incrementa. Por esta razón utilizaremos ésta variable para rellenar los valores cero en esta columna. En el caso de la variable `income_type` no la podremos utilizar ya que la mayor parte de valores anómalos se encontraban en la categoría `retiree`.

Si observamos la tabla `days_by_age`, existe una diferencia significativa entre la media y mediana, así que rellenaremos nuestros valores cero con la mediana. Para esto escribiremos una función `median_by_groups`que calcule las medianas de acuerdo al grupo de edad. Luego, crearemos una nueva columna `median_by_age`, a partir de la cual reemplazaremos los valores cero con el valor de la mediana equivalente al grupo de edad. 

In [63]:
def median_by_groups(row):
    age_group = row['age_group']
    return days_by_age['median'][age_group]

In [64]:
# Comprobamos si la función funciona correctamente
median_by_groups(data.loc[10])

1589.0

In [65]:
data['median_by_age'] =  data.apply(median_by_groups, axis = 1)

In [66]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,median_by_age
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-49,2009.0
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,30-39,1589.0
2,0,5623,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,30-39,1589.0
3,3,4124,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,30-39,1589.0
4,0,0,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,50-59,2261.0


In [67]:
# Reemplazar valores cero
data.loc[data['days_employed'] == 0, 'days_employed'] = data['median_by_age']

Ya que reemplazamos los valores NaN y anómalos de la columna days_employed por cero, vamos a comprobar que ya no se encuentren presentes estos valores en nuestro dataframe, para ello realizamos una indexación lógica y efectivamente se comprueba que los valores cero han sido correctamente rellenados. Se vuelve a llamar al método info para confirmar que no existen valores ausentes en nuestra tabla.

In [68]:
# Se comprueba si ya no existen valores cero en nuestro dataframe
data.loc[data['days_employed'] == 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,median_by_age


In [69]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     21454 non-null  float64
 2   dob_years         21454 non-null  int64  
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      21454 non-null  float64
 11  purpose           21454 non-null  object 
 12  age_group         21454 non-null  object 
 13  median_by_age     21454 non-null  float64
dtypes: float64(3), int64(5), object(6)
memory usage: 2.3+ MB


## Clasificación de datos <a class="anchor" id="capitulo6"></a>

Para responder a las cuatro hipótesis planteadas en la introducción de este proyecto, vamos a clasificar los datos para poder interpretar los resultados de una manera más precisa. Las hipótesis están enfocadas en que el número de hijos, estado civil, ingresos totales y el propósito de un préstamo pueden llegar a afectar el pago a tiempo del mismo. En el caso de las variables número de hijos y estado civil, ya presentan una clasificación o un id que va a permitir claridad al momento de mostrar los resultados. Pero si observamos los datos sobre los ingresos totales y el propósito de un préstamo, los datos no están clasificados y al probar las diferentes hipótesis, estos valores no van a permitir encontrar patrones así que necesitan una correcta clasificación. 

Vamos a empezar por revisar los valores que encontramos en la columnas `purpose`.

In [70]:
data['purpose']

0          purchase of the house
1                   car purchase
2          purchase of the house
3        supplementary education
4              to have a wedding
                  ...           
21449       housing transactions
21450          purchase of a car
21451                   property
21452          buying my own car
21453               to buy a car
Name: purpose, Length: 21454, dtype: object

En la columna `purpose` se pueden observar diferentes propósitos para un préstamo, sin embargo algunos de estos datos parecen repetitivos, por ejemplo 'buying my own car' es similar a 'to buy a car', encontes será necesario analizar más a fondo los valores únicos de esta columna.

In [71]:
data['purpose'].value_counts()

wedding ceremony                            791
having a wedding                            768
to have a wedding                           765
real estate transactions                    675
buy commercial real estate                  661
housing transactions                        652
buying property for renting out             651
transactions with commercial real estate    650
purchase of the house                       646
housing                                     646
purchase of the house for my family         638
construction of own property                635
property                                    633
transactions with my real estate            627
building a real estate                      624
buy real estate                             621
purchase of my own house                    620
building a property                         619
housing renovation                          607
buy residential real estate                 606
buying my own car                       

Al aplicar el método value_counts nos encontramos que dentro de los propósitos para un préstamo existen siete grupos principales: matrimonio, bienes raíces, alojamiento, compra de vivienda, construcción de propiedad, compra de un auto y educación. Para poder procesar posteriormente esta información vamos a crear una función `generate_id` que creará un tema común de la columna purpose en base a estos siete grupos previamente identificados.

In [72]:
def generate_id(purpose):
    if 'wedding' in purpose:
        return 'wedding ceremony'
    if 'real estate' in purpose:
        return 'real estate'
    if 'housing' in purpose or 'house' in purpose or 'property' in purpose:
        return 'buy a house'
    if 'car' in purpose:
        return 'buy a car'
    if 'education' in purpose or 'university' in purpose or 'educated' in purpose:
        return 'education'

Se aplica la función `generate_id` a la nueva columna `purpose_id` que contendrá las categorías creadas en base a temas comunes de la columna purpose.

In [73]:
data['purpose_id'] = data['purpose'].apply(generate_id)
data['purpose_id'].value_counts(dropna=False)

buy a house         6347
real estate         4464
buy a car           4306
education           4013
wedding ceremony    2324
Name: purpose_id, dtype: int64

En el caso de los valores de la columna `total_income`, podemos observar un amplio rango de valores que van desde los 3306 a los 362496, así que va a ser necesario clasificarlos para poder responder a nuestras hipótesis.

In [74]:
data['total_income']

0        40620.102
1        17932.802
2        23341.752
3        42820.568
4        25378.572
           ...    
21449    35966.698
21450    24959.969
21451    14347.610
21452    39054.888
21453    13127.587
Name: total_income, Length: 21454, dtype: float64

In [75]:
# Estadísticas de la columna total_income
data['total_income'].agg({'mean', 'median', 'min', 'max'})

max       362496.645000
median     24379.051500
mean       26797.900567
min         3306.762000
Name: total_income, dtype: float64

Para agrupar los datos dentro de la columna `total_income` vamos a considerar la distribución de los datos, la mayoría de datos se encuentran en un valor promedio aproximado de 24000 ingresos mensuales. Los estadísticos recomiendan ignorar cifras extremadamente altas y bajas, y centrarse en cifras de rango medio. Por esta razón, vamos a establecer intervalos de 10000, y aquellos importes mayores a 100000 se englobarán en un intervalo de clase única 'mayor 100000'. Esta clasificación facilitará la visualización de resultados, ya que con intervalos más pequeños se dificulta la interpretación de los resultados. 

Vamos a crear la función `income_id` que permitirá clasificar los ingresos totales en base a los rangos previamente establecidos. La nueva clasificación se guardará en la columna `income_category`. Antes de aplicar la función, se decició convertir la columna `total_income` a valores enteros para evitar valores flotantes no categorizados.

In [76]:
def income_id(income):
    if income < 10000:
        return 'menor 10000'
    if income >= 10000 and income <= 20000:
        return '10000-20000'
    if income >= 20001 and income <= 30000:
        return '20001-30000'
    if income >= 30001 and income <= 40000:
        return '30001-40000'
    if income >= 40001 and income <= 50000:
        return '40001-50000'
    if income >= 50001 and income <= 60000:
        return '50001-60000'
    if income >= 60001 and income <= 70000:
        return '60001-70000'
    if income >= 70001 and income <= 80000:
        return '70001-80000'
    if income >= 80001 and income <= 90000:
        return '80001-90000'
    if income >= 90001 and income <= 100000:
        return '90001-100000'
    if income > 100000:
        return 'mayor 100000'

In [77]:
data['total_income'] = data['total_income'].astype('int')

In [78]:
data['income_category'] = data['total_income'].apply(income_id)

In [79]:
data['income_category'].value_counts(dropna=False)

20001-30000     7660
10000-20000     6450
30001-40000     3605
40001-50000     1492
menor 10000      926
50001-60000      648
60001-70000      294
70001-80000      157
mayor 100000      99
80001-90000       83
90001-100000      40
Name: income_category, dtype: int64

## Comprobación de las hipótesis  <a class="anchor" id="capitulo7"></a>


Para calcular la tasa de incumplimiento se dividió la suma de los valores en la columna `debt` para el número total de valores en la misma. Estos resultados se almacenaron en cuatro tablas dinámicas para permitir una mejor visualización de los resultados. 

**¿Existe una correlación entre tener hijos y pagar a tiempo?**

In [80]:
# Datos sobre el número de hijos y los pagos puntuales
debt_by_children = data.pivot_table(index='children', values='debt', aggfunc=['sum','count'], margins=True)
debt_by_children['payment_rate'] = ((debt_by_children['count']-debt_by_children['sum'])/debt_by_children['count'])*100

# Tasa de incumplimiento en función del número de hijos
debt_by_children['default_rate'] = (debt_by_children['sum']/debt_by_children['count'])*100
debt_by_children

,sum,count,payment_rate,default_rate
,debt,debt,,
children,,,,
0,1063,14091,92.456178,7.543822
1,445,4855,90.834192,9.165808
2,202,2128,90.507519,9.492481
3,27,330,91.818182,8.181818
4,4,41,90.243902,9.756098
5,0,9,100.000000,0.000000
All,1741,21454,91.884963,8.115037


**Conclusión**

En tabla `debt_by_children` se puede observar los resultados con relación a la primera hipótesis planteada, la cual establece que el número de hijos influye en el pago a tiempo de un préstamo. Se puede evidenciar que las tasas de incumplimiento de pago más altas corresponden a las categorías de cuatro hijos (9.75%) y de dos hijos (9.45%), seguidas de las categoría de un hijo con el 9.23% y tres hijos con el 8.18%. A su vez, la tasa de incumplimiento más baja (7.54%) corresponde a aquellos clientes que no tienen hijos. Con estos datos se puede concluir que si bien existe una pequeña diferencia en la tasa de pago a tiempo de acuerdo al número de hijos, ésta no es lo suficientemente significativa como para confirmar que tener o no hijos es un factor determinante en la falta de pago de un préstamo a tiempo. 


**¿Existe una correlación entre la situación familiar y el pago a tiempo?**

In [81]:
# Datos del estado familiar y los pagos a tiempo
debt_by_family =  data.pivot_table(index='family_status', values='debt', aggfunc=['sum', 'count'], margins=True)
debt_by_family['payment_rate'] = ((debt_by_family['count']-debt_by_family['sum'])/debt_by_family['count'])*100

# Tasa de incumplimiento basada en el estado familiar
debt_by_family['default_rate'] = (debt_by_family['sum']/debt_by_family['count'])*100
debt_by_family

,sum,count,payment_rate,default_rate
,debt,debt,,
family_status,,,,
civil partnership,388,4151,90.652855,9.347145
divorced,85,1195,92.887029,7.112971
married,931,12339,92.454818,7.545182
unmarried,274,2810,90.249110,9.750890
widow / widower,63,959,93.430657,6.569343
All,1741,21454,91.884963,8.115037


**Conclusión**

La segunda hipótesis estaba relacionada con el estado civil de un cliente y su relación con la falta de pago de un préstamo. Aquellos clientes casados tienden a no pagar a tiempo sus préstamos. Para probar esta hipótesis vamos a fijarnos en la tabla `debt_by_family`, en ésta la tasa de incumplimiento más alta corresponde a personas solteras, con un 9.75%, seguido de las personas que viven en unión civil con una tasa del 9.34%. Por el contrario, las tasas más bajas son de clientes viudos con un 6.56% y casados con un 7.54%. Se concluye entonces que existen diferencias entre el estado civil y el pago a tiempo de un préstamo, sin embargo las diferencias porcentuales no son tan trascendentales como para afirmar que una persona soltera va a ser un riesgo para un préstamo bancario.



**¿Existe una correlación entre el nivel de ingresos y el pago a tiempo?**

In [82]:
# Datos del nivel de ingresos y los pagos a tiempo
debt_by_income =  data.pivot_table(index='income_category', values='debt', aggfunc=['sum', 'count'], margins=True)
debt_by_income['payment_rate'] = ((debt_by_income['count']-debt_by_income['sum'])/debt_by_income['count'])*100

# Tasa de incumplimiento basada en el nivel de ingresos
debt_by_income['default_rate'] = (debt_by_income['sum']/debt_by_income['count'])*100
debt_by_income

,sum,count,payment_rate,default_rate
,debt,debt,,
income_category,,,,
10000-20000,550,6450,91.472868,8.527132
20001-30000,676,7660,91.174935,8.825065
30001-40000,263,3605,92.704577,7.295423
40001-50000,102,1492,93.163539,6.836461
50001-60000,54,648,91.666667,8.333333
60001-70000,16,294,94.557823,5.442177
70001-80000,8,157,94.904459,5.095541
80001-90000,6,83,92.771084,7.228916


**Conclusión**

La tercera hipótesis busca encontrar una correlación entre el ingreso mensual de un cliente con la tasa de incumplimiento de pago de un préstamo. Si analizamos la tabla `debt_by_income`, las personas con la tasa de incumplimiento más alta (8.68%) corresponden a aquellos con ingresos mensuales entre 20001-30000, seguido de los clientes que ganan entre 10000-20000, con un 8.52%. A su vez, las tasas más bajas de 5% y 5.09% corresponden a los grupos que generan ingresos entre 90001-100000 y 70001-80000. También se puede observar que aquellos que generan ingresos superiores a los 100000 no tienen la tasa más baja de incumplimiento (6.06%). Considerando todo esto, se concluye que si bien existen disimilitudes entre el nivel de ingresos de un cliente y la tasa de incumplimiento de un préstamo, éstas no son tan significativas como para afirmar que se evidencia una alta correlación con una sola categoría. 


**¿Cómo afecta el propósito del crédito a la tasa de incumplimiento?**

In [83]:
# Tasa de incumplimiento por cada propósito del crédito
debt_by_purpose =  data.pivot_table(index='purpose_id', values='debt', aggfunc=['sum', 'count'], margins=True)
debt_by_purpose['payment_rate'] = ((debt_by_purpose['count']-debt_by_purpose['sum'])/debt_by_purpose['count'])*100
debt_by_purpose['default_rate'] = (debt_by_purpose['sum']/debt_by_purpose['count'])*100
debt_by_purpose

,sum,count,payment_rate,default_rate
,debt,debt,,
purpose_id,,,,
buy a car,403,4306,90.640966,9.359034
buy a house,446,6347,92.973058,7.026942
education,370,4013,90.779965,9.220035
real estate,336,4464,92.473118,7.526882
wedding ceremony,186,2324,91.996558,8.003442
All,1741,21454,91.884963,8.115037


**Conclusión**

Nuestra cuarta hipótesis establece que el propósito destinado de un préstamo es el que puede afectar la tasa de puntualidad. Es así por ejemplo que en la tabla `debt_by_purpose` se puede observar como aquellos clientes que buscan comprar un auto alcanzan una tasa de imcumplimiento del 9.35%, seguido de las personas que utilizan los préstamos para fines educativos, con un 9.22%. Por otro lado, la tasa más baja del 7.02%, corresponde a las personas que realizaron la compra de una casa seguido de aquellas que invirtieron en bienes raíces, con un 7.52%. Con esto se puede concluir que el propósito de un préstamo si podría influenciar el pago del mismo a tiempo, pero al observar diferencias porcentuales de entre 1-2% entre categoría, éstas no son lo suficientemente relevantes.

# Conclusión general  <a class="anchor" id="conclusion"></a>

***Conclusiones sobre el preprocesamiento de datos***

1. Al realizar un análisis comparativo de las distribuciones de las diferentes variables entre los datos con y sin valores ausentes, se pudo concluir que los datos no presentan un patrón subyacente y se confirmó su aleatoriedad. Además, de que el número de valores ausentes fue del 10% y presentó simetría entre las columnas `days_employed` y `total_income`.
2. Se encontraron datos incorrectos en las columnas `education`, `dob_years`, `gender`, `days_employed` y `children`. En el caso de las columnas `children` y `dob_years`, los valores únicos incorrectos fueron reemplazados por la media. En la columna education los valores duplicados se corrigieron con el método str.lower. En la columna género el objeto incorrecto fue reemplazado por la categoría más común. La columna `days_employed` presentó valores atípicos que fueron transformados en números enteros y absolutos.
3. Los valores ausentes de la columna `total_income` fueron reemplazados por el promedio de acuerdo al nivel de educación y el tipo de ingresos del cliente. En el caso de la columna `days_employed` los valores ausentes se reemplazaron con la mediana según el rango de edad del cliente, esta columna presentó un reto especial ya que se registraron datos con días de trabajo irreales, los cuales estaban relacionados con las categorías unemployed y retiree de la variable tipo de ingreso.
4. Se encontraron 71 datos duplicados, los cuales fueron eliminados por el método drop_duplicates. 


***Conclusiones sobre hipótesis planteadas***

Tras analizar los datos se pudo observar:

1. El número de hijos generó diferencias en las tasas de incumpliento de un préstamo, así aquellos clientes con cuatro hijos presentaron la mayor tasa de incumplimiento, mientras que la tasa más baja correspondió a personas sin hijos.
2. En el caso del estado civil, los clientes con la tasa más alta de incumplimiento corresponde a las personas solteras, en tanto que la tasa más baja pertenece a las personas viudas y casadas.
3. Los clientes con ingresos entre los 10000-30000 presentaron las tasas de incumplimiento más altas, por otro lado las personas con ingresos entre 90001-10000 registraron la tasa más baja. 
4. La búsqueda de educación y la compra de un carro tienen las tasas de imcumplimiento más altas, por otro parte la compra de una casa y la inversión en bienes raíces presentaron las tasas más bajas.

En base a esto se puede concluir que el número de hijos, estado civil, nivel de ingresos mensual y el propósito de un préstamo influyeron de cierta manera en la tasa de incumplimiento de un préstamo, sin embargo las diferencias evidenciadas no fueron lo suficientemente significativas para confirmar una correlación fuerte con una sola de las diferentes categorías. Por lo tanto, en relación a nuestras hipótesis se puede establecer que sí existen conexiones entre los parámetros estudiados y el pago de un préstamo, sin embargo dadas las bajas diferencias porcentuales, se requiere de análisis más precisos o pruebas de hipótesis para poder hacer afirmaciones categóricas. 

Finalmente, se recomendaría considerar estos factores al realizar una puntuación de crédito: clientes sin hijos, personas viudas o casadas, ingresos mensuales entre los 90000-100000, y aquellos que buscan comprar una casa o bienes raíces, pero siempre tomando en consideración que no hubo una dependencia mayor a estos factores.